In [2]:
####### V2 for code upside
import os
import re
from typing import List
from langchain.docstore.document import Document
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import MarkdownHeaderTextSplitter

def hierarchical_markdown_split(md_text: str, path_prefix: str = "") -> list[Document]:
    splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[
        ("#", "title"),
        ("##", "section"),
        ("###", "subsection"),
        ("####", "subsubsection")
    ])
    docs = splitter.split_text(md_text)

    result_docs = []
    current_title = None
    for doc in docs:
        metadata = doc.metadata
        if "title" in metadata:
            current_title = metadata["title"]

        if current_title:
            full_title = ""
            # full_title = f"[{current_title}]"
            # if "section" in metadata:
            #     full_title += f" > {metadata['section']}"
            if "subsection" in metadata:
                full_title += f" > {metadata['subsection']}"
            if "subsubsection" in metadata:
                full_title += f" > {metadata['subsubsection']}"

            content = f"[{full_title}]\n\n{doc.page_content}"
            doc = Document(page_content=content, metadata=doc.metadata)

        result_docs.append(doc)

    return result_docs

def load_markdown_file(file_path: str) -> str:
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()
      
str_md_file = load_markdown_file("data/dev_center_guide_touched.md")

docs = hierarchical_markdown_split(str_md_file)

print('split docs count :', len(docs))
for doc in docs:
    print(doc.page_content)
    print("--------------------------------")
    print(doc.metadata)  # Print the first 1000 characters of each document
    print("====================================")


split docs count : 58
[]

원스토어 인앱결제는 다양한 결제수단과 강력한 보안 그리고 편리하고 안전한 결제 서비스를 제공합니다.  
- 최신버전 인앱결제 가이드
[원스토어 인앱결제 API V7 (SDK V21) 안내 및 다운로드](#원스토어-인앱결제-api-v7sdk-v21-연동-안내-및-다운로드)
--------------------------------
{'title': '원스토어 인앱 결제 연동 가이드', 'section': '원스토어 In-App SDK'}
[]

원스토어의 최신 인앱결제 API V7(SDK V21)이 출시되었습니다.
보다 강력하고 다양한 기능을 지원하는 최신 버전을 적용해보세요.  
- API V4(SDK V16) 이하 버전과는 호환되지 않습니다. 인앱결제 API V4(SDK V16)에 대한 안내 및 다운로드는 여기를 클릭해주세요.
- 현재 판매중인 앱을 대한민국 외 국가/지역으로 배포하기 위해서는 아래 가이드를 참고해주세요
- [대한민국 외 국가 및 지역 배포를 위한 가이드](https://onestore-dev.gitbook.io/dev/tools/glb)  
If you are comfortable with English, please change the language to English from the upper left side in this page.
--------------------------------
{'title': '원스토어 인앱 결제 연동 가이드', 'section': '원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드'}
[ > 01. 원스토어 인앱결제 개요 > 원스토어 인앱결제란?]

원스토어 인앱결제(In-App Purchase, IAP)는 안드로이드 앱 내에 구현된 상품을 원스토어의 인증 및 결제 시스템을 이용하여 사용자에게 판매, 청구하여 개발자에게 정산하는 서비스입니다.  
인앱 상품 결제를 위해 원스토어 서비스(ONE store service, OSS) 앱과 

In [ ]:
# create function blocks
import os
import re
from pathlib import Path
from langchain_core.documents import Document

def extract_package(content: str) -> str:
    match = re.search(r'^\s*package\s+([\w.]+)', content, re.MULTILINE)
    return match.group(1) if match else ""

def extract_code_blocks(content: str) -> list[tuple[str, str]]:
    # 클래스/객체/함수 단위로 의미 블록 추출
    pattern = r"""
        (
            (/\*\*[\s\S]*?\*/)?      # optional javadoc
            \s*
            (class|object|fun)       # declaration
            \s+
            (\w+)                    # symbol name
            [^{]*\{                  # signature up to opening brace
        )
    """
    matches = list(re.finditer(pattern, content, re.MULTILINE | re.VERBOSE))
    blocks = []

    for i, match in enumerate(matches):
        start = match.start()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(content)
        block_text = content[start:end].strip()
        symbol = match.group(4)
        blocks.append((symbol, block_text))

    return blocks

def load_kotlin_documents(project_root: str) -> list[Document]:
    project_root_path = Path(project_root)
    documents = []

    for file_path in project_root_path.rglob("*.kt"):
        rel_path = file_path.relative_to(project_root_path)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        package = extract_package(content)
        blocks = extract_code_blocks(content)

        for symbol, block in blocks:
            doc = Document(
                page_content=block,
                metadata={
                    "source": str(rel_path),
                    "symbol": symbol,
                    "package": package,
                    "type": "code",
                    "language": "kotlin"
                }
            )
            documents.append(doc)

    return documents
  

docs = load_kotlin_documents("code_sample/onestore_iap_release/")
print(f"{len(docs)}개 문서를 생성하였습니다.")
print(docs[0].metadata)

In [ ]:
## create class blocks only
def extract_class_blocks(content: str) -> list[tuple[str, str]]:
    pattern = r"""
        (
            (/\*\*[\s\S]*?\*/)?         # optional javadoc
            \s*
            (class|object)              # type
            \s+
            (\w+)                       # name
            [^{]*\{                    # open brace
        )
    """
    matches = list(re.finditer(pattern, content, re.MULTILINE | re.VERBOSE))
    blocks = []

    for i, match in enumerate(matches):
        start = match.start()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(content)
        block_text = content[start:end].strip()
        symbol = match.group(4)
        blocks.append((symbol, block_text))

    return blocks

def load_kotlin_class_documents(project_root: str) -> list[Document]:
    project_root_path = Path(project_root)
    documents = []

    for file_path in project_root_path.rglob("*.kt"):
        rel_path = file_path.relative_to(project_root_path)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        package = extract_package(content)
        # blocks = extract_code_blocks(content)
        blocks = extract_class_blocks(content)
        
        for symbol, block in blocks:
            doc = Document(
                page_content=block,
                metadata={
                    "source": str(rel_path),
                    "symbol": symbol,
                    "package": package,
                    "type": "code",
                    "language": "kotlin",
                    "scope": "class"  # or "object" based on your needs 
                }
            )
            documents.append(doc)

    return documents

docs_with_only_classes = load_kotlin_class_documents("code_sample/onestore_iap_release/")
print(f"{len(docs_with_only_classes)}개 문서를 생성하였습니다.")
for doc in docs_with_only_classes[:3]:
    print(doc.metadata)  # Print the metadata of the first 3 documents
    print('--------------------')
    print(doc.page_content)  # Print the first 100 characters of each document's content
    print('====================')

In [ ]:
def embed_and_save(docs: List[Document], output_path: str):
    # embedding_model = OllamaEmbeddings(model="eeve-kor-10.8-KM:latest")
    embedding_model = OllamaEmbeddings(model="exaone3.5:latest")
    db = FAISS.from_documents(docs, embedding_model)
    db.save_local(output_path)
    print(f"✅ 저장 완료: {output_path}")

str_md_file = load_markdown_file("data/dev_center_guide_touched.md")
print('loaded data length :', len(str_md_file))

# docs_markdown = header_splitter.split_text(str_md_file)
docs_markdown = hierarchical_markdown_split(str_md_file)
# docs_with_only_classes = load_kotlin_class_documents("code_sample/onestore_iap_release/")
# docs_with_functions = load_kotlin_documents("code_sample/onestore_iap_release/")

print(f"{len(docs_markdown)}개 문서(from md)를 생성하였습니다.")
# print(f"{len(docs_with_only_classes)}개 문서(from code)를 생성하였습니다.")

total_docs = docs_markdown #+ docs_with_only_classes + docs_with_functions
print(f"총 {len(total_docs)}개의 문서를 생성하였습니다.")

embed_and_save(total_docs, "models/faiss_vs_rag_iap_touched_v6")


In [2]:
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate


# ✅ 3. 임베딩 모델 초기화 (Ollama)
embedding_model = OllamaEmbeddings(model="exaone3.5:latest")
# embedding_model = OllamaEmbeddings(model="eeve-kor-10.8-KM:latest")

# 저장된 데이터를 로드
loaded_db = FAISS.load_local(
    folder_path="models/faiss_vs_rag_iap_touched_v6",
    # index_name="index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True,
)

retriever = loaded_db.as_retriever(
    # search_type="mmr",
    search_type="similarity",
    search_kwargs={"k": 10, "fetch_k": 10, "lambda_mult": 0.6}
)

res = retriever.invoke(
    "Push Notification Service(PNS)는 무엇입니까?"
)

print(f"검색된 문서 수: {len(res)}")

for doc in res: 
    print('--------------------')
    print(doc.page_content)  # Print first 100 characters of each document
    print(doc.metadata)


/var/folders/lr/mv7qs1n11s58s4mq9t7vlxr40000gn/T/ipykernel_97673/1183484649.py:15: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_model = OllamaEmbeddings(model="exaone3.5:latest")


검색된 문서 수: 10
--------------------
[ > 02. 사전준비 > 라이선스 키(Public Key) 및 OAuth 인증 정보 확인하기]

공통정보 >  라이선스 관리 메뉴에서 라이선스 키와 서버 API를 위한 OAuth 인증 정보를 확인할 수 있습니다.  
- 라이선스 키 : 원스토어가 전달한 인앱결제 내역의 위변조 여부를 확인하는 용도로 사용합니다.
- OAuth 인증 정보 : 원스토어 서버 API를 사용하기 위한 인증 용도로 사용합니다.
{'title': '원스토어 인앱 결제 연동 가이드', 'section': '원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드', 'subsection': '02. 사전준비', 'subsubsection': '라이선스 키(Public Key) 및 OAuth 인증 정보 확인하기'}
--------------------
[ > 07. PNS (Push Notification Service) 이용하기 > PNS 수신 서버 URL 설정]

PNS를 수신 받을 개발사 서버의 URL은 '개발자센터 > Apps > 상품 선택 > In-App정보' 메뉴에서 'PNS 관리' 버튼을 클릭하면 설정할 수 있습니다.
URL은 Sandbox(개발용) 결제환경 및 상용(상용테스트 포함) 결제환경을 각각 설정할 수 있으며, 개발용/상용 서버가 동일할 경우 동일한 URL을 입력하시면 됩니다.
{'title': '원스토어 인앱 결제 연동 가이드', 'section': '원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드', 'subsection': '07. PNS (Push Notification Service) 이용하기', 'subsubsection': 'PNS 수신 서버 URL 설정'}
--------------------
[ > 06. 원스토어 인앱결제 서버 API (API V7) > 개요]

원스토어 인앱결제 서버 API란 원스토어에서 결제된 인앱 상품의 데이터를 조회하거나 결제 상태를 

In [3]:
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate


# ✅ 3. 임베딩 모델 초기화 (Ollama)
embedding_model = OllamaEmbeddings(model="exaone3.5:latest")
# embedding_model = OllamaEmbeddings(model="eeve-kor-10.8-KM:latest")
# 저장된 데이터를 로드
loaded_db = FAISS.load_local(
    folder_path="models/faiss_vs_rag_iap_touched_v6",
    # index_name="index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True,
)

retriever = loaded_db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 10, "fetch_k": 25, "lambda_mult": 0.6}
)

# search_result = loaded_db.similarity_search("정기결제 코드를 작성해주세요", k=10)
# for doc in search_result:
#     print('--------------------')
#     print(doc.page_content)
#     print(doc.metadata)

prompt = PromptTemplate.from_template(
    """
당신은 한국어로 된 질문에 답변하는 AI입니다.
주어진 질문에 대해 관련된 문서에서 정보를 찾아 답변을 작성하세요.
답변을 생성할 수 없을 경우 '해당 질의에 대한 답변을 찾을 수 없습니다.'라고 응답하세요.
해당 질문은 개발자들이 하는 질문으로 최대한 개발자 친화적인 답변을 생성해주세요.
가능한한 코드 예시를 포함하여 답변을 작성해주세요.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)

llm = ChatOllama(
    # model="eeve-kor-10.8-KM:latest",
    model="exaone3.5:latest",
    # model="mixtral:latest",  # 필요 시 조절
    temperature=0.0  # 필요 시 조절
)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

question = "인앱결제시 단말에서 발생하는 내부적인 데이터 플로우를 알려주세요." 
response = chain.invoke(question)
import sys
sys.stdout.write(response)
    

/var/folders/lr/mv7qs1n11s58s4mq9t7vlxr40000gn/T/ipykernel_97673/3473349609.py:52: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(


인앱 결제 시 단말에서 발생하는 내부적인 데이터 플로우는 다음과 같이 요약될 수 있습니다. 이 플로우는 주로 사용자 인터페이스에서 시작하여 결제 처리, 그리고 서버와의 통신을 포함합니다. 특히 원스토어 인앱 결제 시스템을 기반으로 설명하겠습니다.

### 1. 사용자 인터페이스 단계
- **사용자 요청**: 사용자가 앱 내에서 상품을 선택하고 결제를 요청합니다.
- **UI 표시**: 앱은 원스토어 인앱 결제 SDK를 통해 결제 UI를 표시합니다. 이 UI는 팝업 또는 전체 화면 결제 창으로 구성될 수 있습니다 (`전면/팝업 결제화면 선택 가능` 참조).

### 2. 앱 내부 처리 단계
- **결제 요청 생성**: 사용자가 결제 정보를 입력하면 앱은 이 정보를 기반으로 결제 요청을 생성합니다.
  ```java
  // 예시 코드: 원스토어 인앱 결제 요청 생성
  OneStoreIapClient client = OneStoreIapClient.getInstance();
  PurchaseParams params = new PurchaseParams.Builder()
      .setProductId("your_product_id")
      .setToken("user_payment_token") // 사용자 인증 토큰
      .build();
  
  client.purchase(context, params, new PurchaseCallback() {
      @Override
      public void onPurchaseSuccess(PurchaseResult result) {
          // 결제 성공 처리 로직
      }

      @Override
      public void onPurchaseFailure(PurchaseFailureResult failureResult) {
          // 결제 실패 처리 로직
      }
  });
  ```

### 3. 서버 통신 단계
- **결제 요청 전송**: 앱

2078

In [ ]:
question = "PNS가 무엇이고 어떤 경우에 사용하는지 알려주세요." 
response = chain.invoke(question)
import sys
sys.stdout.write(response)